In [ ]:
import requests
import pandas as pd

api_key = 'b1675576169e74c528a4012caa68ea'
cities = ['London', 'Paris', 'New York', 'Tokyo', 'Mumbai']

weather_data = []

for city in cities:
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = response.json()
    
    if data['cod'] == 200:
        city_data = {
            'City': city,
            'Temp': data['main']['temp'],
            'Temp_min': data['main']['temp_min'],
            'Temp_max': data['main']['temp_max'],
            'Humidity': data['main']['humidity']
        }
        weather_data.append(city_data)

# Convert to DataFrame for easy manipulation
weather_df = pd.DataFrame(weather_data)
print(weather_df)


Empty DataFrame
Columns: []
Index: []


In [11]:
import requests
import pandas as pd

api_key = '0620a1b60e76e33618926886b9c616c'  # Make sure this is correct
cities = ['London', 'Paris', 'New York', 'Tokyo', 'Mumbai']

weather_data = []

for city in cities:
    # Corrected URL formatting with f-strings
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    
    # Print the URL to verify it's correct
    print(f"Fetching data from: {url}")
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        if data.get('cod') == 200:
            city_data = {
                'City': city,
                'Temp': data['main']['temp'],
                'Temp_min': data['main']['temp_min'],
                'Temp_max': data['main']['temp_max'],
                'Humidity': data['main']['humidity']
            }
            weather_data.append(city_data)
        else:
            print(f"Error with city {city}: {data.get('message')}")
    else:
        print(f"Failed to fetch data for {city}, Status code: {response.status_code}")

weather_df = pd.DataFrame(weather_data)

if not weather_df.empty:
    print(weather_df)
else:
    print("No weather data fetched. Please check the API responses.")


Fetching data from: http://api.openweathermap.org/data/2.5/weather?q=London&appid=0620a1b60e76e33618926886b9c61c6c&units=metric
Failed to fetch data for London, Status code: 401
Fetching data from: http://api.openweathermap.org/data/2.5/weather?q=Paris&appid=0620a1b60e76e33618926886b9c61c6c&units=metric
Failed to fetch data for Paris, Status code: 401
Fetching data from: http://api.openweathermap.org/data/2.5/weather?q=New York&appid=0620a1b60e76e33618926886b9c61c6c&units=metric
Failed to fetch data for New York, Status code: 401
Fetching data from: http://api.openweathermap.org/data/2.5/weather?q=Tokyo&appid=0620a1b60e76e33618926886b9c61c6c&units=metric
Failed to fetch data for Tokyo, Status code: 401
Fetching data from: http://api.openweathermap.org/data/2.5/weather?q=Mumbai&appid=0620a1b60e76e33618926886b9c61c6c&units=metric
Failed to fetch data for Mumbai, Status code: 401
No weather data fetched. Please check the API responses.


In [ ]:
import pyodbc

# SQL Server connection details
server = 'your_server'
database = 'your_database'
username = 'your_username'
password = 'your_password'

# Connect to the database
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
cursor = conn.cursor()

# Create the table if not already created
cursor.execute('''
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='WeatherData' AND xtype='U')
    CREATE TABLE WeatherData (
        City VARCHAR(50),
        Temp FLOAT,
        Temp_min FLOAT,
        Temp_max FLOAT,
        Humidity INT
    )
''')
conn.commit()

# Insert the weather data into the table
for _, row in weather_df.iterrows():
    cursor.execute('''
        INSERT INTO WeatherData (City, Temp, Temp_min, Temp_max, Humidity)
        VALUES (?, ?, ?, ?, ?)
    ''', row['City'], row['Temp'], row['Temp_min'], row['Temp_max'], row['Humidity'])
conn.commit()

# Close the connection
cursor.close()
conn.close()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the figure size for the plots
plt.figure(figsize=(12, 6))

# Create a subplot for temperature data
plt.subplot(1, 2, 1)
sns.barplot(x='City', y='Temp', data=weather_df, color='b')
plt.title('Current Temperature for Each City')

# Create a subplot for humidity data
plt.subplot(1, 2, 2)
sns.barplot(x='City', y='Humidity', data=weather_df, color='g')
plt.title('Humidity for Each City')

# Show the plots
plt.tight_layout()
plt.show()


In [2]:
import requests
import pyodbc
import seaborn as sns
import matplotlib.pyplot as plt

api_key = 'b1675576169e74c528a4012caa68eac9'
cities = ['London', 'Paris', 'New York', 'Tokyo', 'Mumbai']

# Step 1: Fetch weather data from OpenWeatherMap API
weather_data = []

for city in cities:
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = response.json()
    
    city_data = {
        'City': city,
        'Temp': data['main']['temp'],
        'Temp_Min': data['main']['temp_min'],
        'Temp_Max': data['main']['temp_max'],
        'Humidity': data['main']['humidity']
    }
    
    weather_data.append(city_data)

# Step 2: Store data in SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=your_server;DATABASE=your_database;UID=your_user;PWD=your_password')
cursor = conn.cursor()

# Create WeatherData table if not exists
cursor.execute("""
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='WeatherData' AND xtype='U')
CREATE TABLE WeatherData (
    City VARCHAR(50),
    Temp FLOAT,
    Temp_Min FLOAT,
    Temp_Max FLOAT,
    Humidity INT
)
""")
conn.commit()

# Insert weather data into the table
for city_data in weather_data:
    cursor.execute("INSERT INTO WeatherData (City, Temp, Temp_Min, Temp_Max, Humidity) VALUES (?, ?, ?, ?, ?)",
                   city_data['City'], city_data['Temp'], city_data['Temp_Min'], city_data['Temp_Max'], city_data['Humidity'])

conn.commit()
cursor.close()
conn.close()

# Step 3: Visualization using Seaborn
# Convert weather data into a DataFrame
import pandas as pd
df = pd.DataFrame(weather_data)

# Create a figure and axes for the plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Variation of Temp, Temp_Min, and Temp_Max for each city
sns.barplot(x='City', y='Temp', data=df, ax=axes[0, 0])
axes[0, 0].set_title('Current Temperature (Temp)')

sns.barplot(x='City', y='Temp_Min', data=df, ax=axes[0, 1])
axes[0, 1].set_title('Minimum Temperature (Temp_Min)')

sns.barplot(x='City', y='Temp_Max', data=df, ax=axes[1, 0])
axes[1, 0].set_title('Maximum Temperature (Temp_Max)')

# Plot 2: Humidity for each city
sns.barplot(x='City', y='Humidity', data=df, ax=axes[1, 1])
axes[1, 1].set_title('Humidity')

# Show the plots
plt.tight_layout()
plt.show()


KeyError: 'main'

In [ ]:
import pyodbc
import pandas as pd

data = {
    "City": ["London", "Paris", "New York"],
    "Current_Temp": [59.0, 60.8, 62.6],
    "Min_Temp": [55.4, 57.2, 59.0],
    "Max_Temp": [62.6, 64.4, 66.2],
    "Humidity": [80, 75, 70]
}
df = pd.DataFrame(data)

conn_str = (
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"CB_DATABASE_VARUN;"  
    r"DATABASE=DATABASE;"  
    r"Trusted_Connection=yes;"
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

check_table_sql = """
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='WeatherData' AND xtype='U')
CREATE TABLE WeatherData (
    City VARCHAR(50),
    Current_Temp FLOAT,
    Min_Temp FLOAT,
    Max_Temp FLOAT,
    Humidity INT
)
"""
cursor.execute(check_table_sql)

insert_sql = """
INSERT INTO WeatherData (City, Current_Temp, Min_Temp, Max_Temp, Humidity)
VALUES (?, ?, ?, ?, ?)
"""

for _, row in df.iterrows():
    cursor.execute(insert_sql, row['City'], row['Current_Temp'], row['Min_Temp'], row['Max_Temp'], row['Humidity'])

conn.commit()
conn.close()


OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

In [13]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
API_KEY = "953a38fd692cdde1d5b8b0f00719654e"
CITIES = ["London", "New York", "Tokyo", "Sydney", "Mumbai"]
weather_data = {"City": [], "Current Temperature (°F)": [], "Min Temperature (°F)": [], "Max Temperature (°F)": [], "Humidity (%)": []}
for city in CITIES:
    URL = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}"
    response = requests.get(URL)
    if response.status_code == 200:
        data = response.json()
        if "main" in data:
            temp = (data["main"]["temp"] - 273.15) * 9/5 + 32
            temp_min = (data["main"]["temp_min"] - 273.15) * 9/5 + 32
            temp_max = (data["main"]["temp_max"] - 273.15) * 9/5 + 32
            humidity = data["main"]["humidity"]

            weather_data["City"].append(city)
            weather_data["Current Temperature (°F)"].append(temp)
            weather_data["Min Temperature (°F)"].append(temp_min)
            weather_data["Max Temperature (°F)"].append(temp_max)
            weather_data["Humidity (%)"].append(humidity)
        else:
            print(f"Error: 'main' key not found in the API response for {city}.")
    else:
        print(f"Error: Failed to fetch data for {city}. HTTP Status Code: {response.status_code}")


df = pd.DataFrame(weather_data)
plt.figure(figsize=(15, 8))

plt.subplot(1, 2, 1)
bar_width = 0.2
positions = range(len(CITIES))
print(positions)
plt.bar([p - bar_width for p in positions], df["Current Temperature (°F)"], width=bar_width, label="Current Temp (°F)", color="blue")
plt.bar(positions, df["Min Temperature (°F)"], width=bar_width, label="Min Temp (°F)", color="orange")
plt.bar([p + bar_width for p in positions], df["Max Temperature (°F)"], width=bar_width, label="Max Temp (°F)", color="green")

plt.xticks(positions, df["City"], rotation=45)
plt.title("Temperature Comparison")
plt.ylabel("Temperature (°F)")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(df["City"], df["Humidity (%)"], marker="o", linestyle="--", color="red", label="Humidity (%)")
plt.title("Humidity Levels")
plt.ylabel("Percentage")
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

import pymysql
import pandas as pd


DATABASE = 'CB_DATABASE'
USERNAME = 'root'
PASSWORD = 'root'


connection = pymysql.connect(
    host='localhost',  
    database=DATABASE,
    user=USERNAME,
    password=PASSWORD
)
cursor = connection.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS weather_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    city VARCHAR(50),
    current_temp FLOAT,
    min_temp FLOAT,
    max_temp FLOAT,
    humidity INT
);
'''
cursor.execute(create_table_query)

insert_query = '''
INSERT INTO weather_data (city, current_temp, min_temp, max_temp, humidity)
VALUES (%s, %s, %s, %s, %s);
'''
for _, row in df.iterrows():
    cursor.execute(
        insert_query,
        (row['City'], 
         row['Current Temperature (°F)'], 
         row['Min Temperature (°F)'], 
         row['Max Temperature (°F)'], 
         row['Humidity (%)'])
    )

connection.commit()
print("Weather data has been successfully loaded into the database.")
cursor.close()
connection.close()


AttributeError: module 'matplotlib' has no attribute 'backends'

Error in callback <function _enable_matplotlib_integration.<locals>.configure_once at 0x000001A73FC4F240> (for post_run_cell), with arguments args (<ExecutionResult object at 1a71dfffc20, execution_count=13 error_before_exec=None error_in_exec=module 'matplotlib' has no attribute 'backends' info=<ExecutionInfo object at 1a73f873c80, raw_cell="import requests
import pandas as pd
import matplot.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/gupta/OneDrive/Desktop/de%20practical/de.ipynb#W3sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AttributeError: module 'matplotlib' has no attribute 'backends'